## Bulk update maps for Use in Collector / Field Maps

This notebook allows the user to define a search string for webmaps and set the ability to "Use in Field Maps" and/or "Use in Collector". It will first search for maps based on a string you define, confirm these are the maps you'd like to update, and then update each for Collector / Field Maps use based on parameters you define. Additionally, this notebook can be imported in ArcGIS Online and used as a hosted notebook.

In [ ]:
from arcgis.gis import GIS
from arcgis.mapping import WebMap
gis = GIS("home")
# gis = GIS("https://arcgis.com","<insert_user_here>", verify_cert=False) 

### Please set the parameters below. Example parameters have been set for this notebook. Note that we will only search for maps we own

In this example query, we'll disable in Field Maps all webmaps that are currently not enabled for Collector. This will prevent field workers from accidentally using the wrong map!

For more on writing this search query, please see: https://doc.arcgis.com/en/arcgis-online/reference/advanced-search.htm

In [ ]:
# this takes "enabled", "disabled", or "no_change"
field_maps_usage = "enabled"
collector_usage = "no_change"
webmap_search_query = "Hydrant NOT typekeywords: 'Collector' OR typekeywords: 'CollectorDisabled'"

# Example query which would only update maps currently enabled for Collector
# webmap_search_query = "title:Hydrant AND typekeywords:'Collector'"

# Example query which would update maps currently enabled for Field Maps
# webmap_search_query = "title:Hydrant AND NOT typekeywords:'FieldMapsDisabled'"

valid_entries = ["enabled", "disabled", "no_change"]
if not field_maps_usage in valid_entries or not collector_usage in valid_entries:
    raise Exception("Check your usage parameters are set correctly")

### Now, let's search for the webmaps that we're going to update

In [ ]:
webmap_search_query = webmap_search_query + f" AND owner:{gis.users.me.username}"
webmaps = gis.content.search(webmap_search_query, item_type='Web Map', max_items=1000, outside_org=False)
print("Webmaps that will be changed: ")
for wm in webmaps:
    display(wm)
print("\n")
if len(webmaps) > 0:
        yn = (input(str(len(webmaps)) + ' web maps found. Are you sure you want to work on all these web maps? (y/N): ') or 'n').lower().strip()
        sure = yn in ['yes', 'y', 'ye']
else:
    raise Exception("No items found with query:", webmap_search_query)

### If you want to continue, we'll now update the keywords needed to use Collector and Field Maps

Note here that we update the map once - once by calling `update()` on the item and once by calling `update()` on the WebMap. The second update call exists to scrub out keywords. So it's possible that your keywords are unchanged despite the modified date of the webmap changing.

In [ ]:
if not sure:
    raise Exception("Script terminated. Please check your search string again")
else:
    for wm_item in webmaps:
        print("Checking whether to update webmap for Field Maps and Collector: " + wm_item.title)
        current_keywords = set(wm_item.typeKeywords)
        if field_maps_usage == "enabled":
            current_keywords.discard("FieldMapsDisabled")
        elif field_maps_usage == "disabled":
            current_keywords.add("FieldMapsDisabled")

        if collector_usage == "enabled":
            current_keywords.discard("CollectorDisabled")
        elif collector_usage == "disabled":
            current_keywords.discard("Collector")
            current_keywords.add("CollectorDisabled")

        if sorted(list(current_keywords)) != sorted(wm_item.typeKeywords):
            print("New keyword added for " + wm_item.title + ". Updating webmap")
            try:
                wm_item.update(item_properties={"typeKeywords": list(current_keywords)})
            except Exception as e:
                print(e)
            wm = WebMap(wm_item)
            # remove bad Collector typekeywords
            wm.update()
        else:
            print("Skipped updating webmap for " + wm_item.title)
        print("\n")
            
print("Done!")